# Wisdom Tree Assignment

$$\text{Implied Roll Yield}_i = \bigg( \frac{\text{Price of contract}_{i-1}}{\text{Price of contract}_{i}} - 1 \bigg) \frac{1}{\text{# of months between the contracts}}$$

In [1]:
# check missing business days
pd.bdate_range(start = i[1].index[0], end = i[1].index[-1] ).difference(i[1].index)

NameError: name 'pd' is not defined

todo weights proportional to roll yield